# DealInfos_Word2Vec feature를 ES에 벌크 인서트

-  환경: ElasticSearch 6
-  테스트 머신: 장민 PC

```
PUT /dealinfos
{
mapping_dealinfos_word2vec.json
}
```


In [1]:
import requests
import time
import urllib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import json
import elasticsearch
import csv
import pickle
from elasticsearch.helpers import bulk
import re
import glob
import os
from datetime import timezone, timedelta, datetime
from pymongo import MongoClient
import random
import numpy as np
import sys
import pandas as pd
import csv

In [2]:
def make_documents(file_path):
    pred_dic = {}
    with open(file_path, 'rb') as jsonfile:
        for row in jsonfile:
            source_ = eval(row)
            source = {}
            source['v'] = source_['v']
            source['words'] = source_['words']
            source['values'] = source_['vectorizedWords']['values']
            doc =  {
                "_op_type":'create',
                "_index":"deal_word2vec",
                "_type":"doc",
                "_id": source['v'],
                "_source": source
            }
            yield(doc)

In [3]:
### Jangmin's PC
es_url = 'localhost:9200'

In [4]:
es = elasticsearch.Elasticsearch(es_url)

In [5]:
es.indices.get_mapping(index='deal_word2vec')

{'deal_word2vec': {'mappings': {'doc': {'properties': {'v': {'type': 'long'},
     'values': {'type': 'double'},
     'words': {'type': 'keyword'}}}}}}

# bulk inserting to ES

In [8]:
data_dir = r'/Users/jangmino/tensorflow/work/wepick_data/w2vec.json'
start = time.time()
total_time = 0
bulk(es, make_documents(data_dir))
total_time += (time.time() - start)
print("Elapsed Time {} sec".format(total_time))

Elapsed Time 1301.6904072761536 sec


In [60]:
res = es.search(index='deal_word2vec', body={'query': {'match':{'v':'2668602'}}})

In [61]:
res['hits']['total']

1

In [64]:
type(res['hits']['hits'][0]['_source']['v'])


int

### wepick_data

format:
deal_id, user_id, slot_id, timestamp, impression_time, label(always one)

In [6]:
data_path = r'd:\WMIND\temp\pos_dataset.csv'

In [26]:
types = {"v":'str', "u":"str", "slot":"str", "rgtime":"str", "imp_dt":"str", "label":"str"}
df = pd.read_csv(data_path, header=None, names=["v", "u", "slot", "rgtime", "imp_dt", "label"])

KeyboardInterrupt: 

In [81]:
def make_wepick_documents(df):
    for u, hist in df.groupby('u'):
        source = {}
        vec = []
        source['u'] = int(u)
        for i in range(len(hist.index)):
            data = dict(hist.iloc[i])
            data['rgtime'] = int(data['rgtime'])
            data['v'] = int(data['v'])
            data['slot'] = int(data['slot'])
            #data['rgtime'] = datetime.fromtimestamp(data['rgtime'])
            del data['u']
            del data['label']
            vec.append(data)
        source['hist'] = vec
        doc = {
            "_op_type":'create',
            "_index":"wepick_seq",
            "_type":"doc",
            "_id": u,
            "_source": source
        }
        yield(doc)

In [7]:
def make_wepick_documents2(file_path):
    with open(file_path) as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            source = {}
            v, u, slot, imp_dt = int(row[0]), int(row[1]), int(row[2]), row[4]         
            source['u'] = u
            source['v'] = v
            source['slot'] = slot
            source['rgtime'] = pd.to_datetime(row[3], unit='ms').to_pydatetime()
            doc = {
                #"_op_type":'create',
                "_index":"wepick_seq",
                "_type":"doc",
                #"_id": u,
                "_source": source
            }
            yield(doc)

In [8]:
for x in make_wepick_documents2(data_path):
    print(x)
    break

{'_index': 'wepick_seq', '_type': 'doc', '_source': {'u': 10000015, 'slot': 24, 'rgtime': datetime.datetime(2018, 3, 29, 10, 16, 2, 191000), 'v': 3473871}}


In [9]:
es.indices.get_mapping(index='wepick_seq')

{'wepick_seq': {'mappings': {'doc': {'properties': {'rgtime': {'type': 'date'},
     'slot': {'type': 'long'},
     'u': {'type': 'long'},
     'v': {'type': 'long'}}}}}}

In [10]:
start = time.time()
total_time = 0
bulk(es, make_wepick_documents2(data_path))
total_time += (time.time() - start)
print("Elapsed Time {} sec".format(total_time))

Elapsed Time 4133.702134132385 sec


In [11]:
res = es.search(index='wepick_seq', body={'query': {'match':{'u':'1000007'}}})

In [13]:
res

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'xjs82GMBXNsJifOTjPYc',
    '_index': 'wepick_seq',
    '_score': 1.0,
    '_source': {'rgtime': '2018-03-13T15:44:10.067000',
     'slot': 4,
     'u': 1000007,
     'v': 3411797},
    '_type': 'doc'},
   {'_id': 'yDs82GMBXNsJifOTjPYc',
    '_index': 'wepick_seq',
    '_score': 1.0,
    '_source': {'rgtime': '2018-03-22T04:43:15.889000',
     'slot': 1,
     'u': 1000007,
     'v': 3447558},
    '_type': 'doc'},
   {'_id': 'yzs82GMBXNsJifOTjPYc',
    '_index': 'wepick_seq',
    '_score': 1.0,
    '_source': {'rgtime': '2018-03-23T13:04:03.368000',
     'slot': 71,
     'u': 1000007,
     'v': 1590568},
    '_type': 'doc'},
   {'_id': '0Ts82GMBXNsJifOTjPYc',
    '_index': 'wepick_seq',
    '_score': 1.0,
    '_source': {'rgtime': '2018-03-29T07:54:16.690000',
     'slot': 1,
     'u': 1000007,
     'v': 3464375},
    '_type': 'doc'},
   {'_id': '1Ds82GMBXNsJifOTjPYc',
    '_index': 'wepick_